<a href="https://colab.research.google.com/github/ytyimin/scm518/blob/main/Pricing_Candy_Tariff_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pricing Candies at SouthCandy

## Objective and Prerequisites

This pricing candies problem shows you how to determine the optimal price of candies when different customers have declining marginal utilities of purchasing multiple pieces of candies. The objectives of the assignment problem are:

* Maximize the overall profit of selling candies to different customers,
* Make sure the customers choose the optimal pieces of candies given any price, and
* Ensure that the customer choices are valid, i.e., each customer segment select only the best choice in terms of pieces of candies to buy.


---
## Problem Description

![picture](https://drive.google.com/uc?id=1JkjnfIMyhl_jiR1fXoiuHIVLxSR6Sf-C)

Most people value the first piece of candy they purchase more than the second piece. They also value the second piece more than the third piece, and so on. How can you take advantage of this when pricing candies? If you charge a single price for each piece of candies, only a few people are going to buy more than one or two pieces. Alternatively, you can try a two-part tariff approach, where you charge an "entry fee" to anyone who buys candies, plus a reduced price per piece purchased. 

For example, if a resonable single price per piece is \\$1.10, then a reasonable two-part tariff might be an entry fee of \\$1.50 and a price of \\$0.50 per piece. This gives some customers an incentive to purchase many pieces of candies. Becuase the cost of purchasing $n$ packs of candies is no longer a linear function of $n$ - it is now piecewise linear. Thus, the two-part tariff is a nonlinear pricing strategy.

The key input is customer sensitivity to price. Rather than having a single demand function, however, here each customer has a unique sensitivity to price. To keep the example small, consider four typical customers from the four market segments where we obtain their willingness to pay for each successive piece of candies. This information is listed in the table below. 

|Pieces	|Customer 1	|Customer 2	|Customer 3	|Customer 4	|
|---|---|---|---|---|
|	1	|	1.24	|	0.92	|	1.27	|	1.49	|
|	2	|	1.03	|	0.85	|	1.11	|	1.24	|
|	3	|	0.89	|	0.69	|	0.96	|	1.10	|
|	4	|	0.80	|	0.58	|	0.85	|	0.97	|
|	5	|	0.77	|	0.50	|	0.73	|	0.81	|
|	6	|	0.66	|	0.43	|	0.63	|	0.71	|
|	7	|	0.59	|	0.36	|	0.51	|	0.63	|
|	8	|	0.51	|	0.32	|	0.45	|	0.53	|
|	9	|	0.42	|	0.26	|	0.39	|	0.42	|
|	10	|	0.35	|	0.22	|	0.32	|	0.35	|

For example, customer 1 is willing to pay \\$1.24 for the first peice, \\$1.03 for the second piece, and only \\$0.35 for the last piece. These four customers are considered representaitve of the four market segments. 

If it cost 0.40 to produce a piece of candy, determine a profit-maximizing single price and two-part tariff. The four market segments ahve 10, 5, 7.5 and 15 (in thousands) customers, repsectively, and that the customers within a market segment all respond identically to price.



## Model Formulation

---



### Indices

$i \in \{1..4\}$: Index to represent customer segments

$j,k \in \{1..10\}$: Index to represent number of candies

### Parameters

$v_{ij}$: Customer segment $i$'s willingness to pay by  for $j$ pieces of candies

$s_{i}$: The size of customer segment $i$

$c$: Unit cost of producing one piece of candy

### Decision Variables

$f$: "Entry fee" to charge for candies.

$p$: Unit price to charge for candies.

$y_{ij}$: Whether customer segment $i$ purchases $j$ pieces of candies.

### Objective Function

- **Profit**. We want to maximize the total profit.


\begin{equation}
\text{Max}_{p,y_{ij}} \quad \sum_{i\in \{1..4\}} \left((f+(p-c)*s_i*j)*\sum_{j\in\{1..10\}}y_{ij}\right)
\tag{0}
\end{equation}

### Constraints

\begin{equation}
\sum_{j \in \{1..10\}} y_{ij} \leq 1 \quad \forall i \in \{1...4\} \quad (\text{Each segement can only select one choice})
\tag{1}
\end{equation}

\begin{equation}
y_{ij}*\left(\sum_{k \in \{1..j\}} v_{ik} - f - p*j\right) \geq \max_{l\in\{1..10\}} \left(\sum_{k \in \{1..l\}} v_{ik} - f - p*l\right) - M*(1-y_{ij}) \quad \forall i \in \{1..4\} \quad (\text{Customer select the best choice})
\tag{2}
\end{equation}

\begin{equation}
y_{ij}*\left(\sum_{k \in \{1..j\}} v_{ik} - f - p*j\right) \geq 0 \quad \forall i \in \{1..4\} \quad (\text{Customer utility must be non-negative if purchasing})
\tag{3}
\end{equation}

\begin{equation}
y_{ij} \in \{0,1\} \quad \forall i,j \quad (\text{Purchase choice binary})
\tag{4}
\end{equation}

\begin{equation}
f,p \geq 0 \quad (\text{Non-negative pricing (can omit)})
\tag{5}
\end{equation}




---

## Python Implementation

We now import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 29.2 MB/s 


In [ ]:
from itertools import product
from math import sqrt, factorial
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0

Set up the inputs

In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('pricing candy')

customer = [*range(0,4)]
candy = [*range(0,10)]

# vij: willingness to pay
v = [[1.24,	1.03, 0.89,	0.80,	0.77,	0.66,	0.59,	0.51,	0.42,	0.35],
     [0.92,	0.85,	0.69,	0.58,	0.50,	0.43,	0.36,	0.32,	0.26,	0.22],
     [1.27,	1.11,	0.96,	0.85,	0.73,	0.63,	0.51,	0.45,	0.39,	0.32],
     [1.49,	1.24,	1.10,	0.97,	0.81,	0.71,	0.63,	0.53,	0.42,	0.35]]

# customer segment size
s = [10,	5,	7.5,	15]

# unit production cost
c = 0.40

# a large number
M = 10


Compute actual charge matrix as well as the set of valid tuples

In [ ]:
# Computing comulative willingness to pay

# initialize an empty matrix to compute cumulative willingness to pay for j pieces of candy
cum = [[0 for j in candy] for i in customer]

# Valid set of tuples
A = []
for i in customer:
    vp = 0
    for j in candy:
        # compute actual charge, taking into account minimum charge
        vp += v[i][j]
        cum[i][j] = vp

        # build valid set of tuples
        tp = i,j
        A.append(tp)

print(np.matrix(cum))    
  

[[1.24 2.27 3.16 3.96 4.73 5.39 5.98 6.49 6.91 7.26]
 [0.92 1.77 2.46 3.04 3.54 3.97 4.33 4.65 4.91 5.13]
 [1.27 2.38 3.34 4.19 4.92 5.55 6.06 6.51 6.9  7.22]
 [1.49 2.73 3.83 4.8  5.61 6.32 6.95 7.48 7.9  8.25]]


Setup decisions, objective, and constraints

In [ ]:
# Build decision variables: price to charge and whether customer segment i selects j pieces of candy
f = m.addVar(vtype=GRB.CONTINUOUS, name='Entry')
p = m.addVar(vtype=GRB.CONTINUOUS, name='Price')
y = m.addVars(A, vtype=GRB.BINARY, name='Purchase')

# add an auxiliary variable that captures the maximum of utility for any given price p
auxU = m.addVars(A, vtype=GRB.CONTINUOUS, lb= - M, name="Utility")
auxUMax = m.addVars(customer, vtype=GRB.CONTINUOUS, lb= -M, name="maxUtility")



In [ ]:
# Objective function: Maximize total profit
m.setObjective(gp.quicksum((f+(p-c)*(j+1))*s[i]*y[(i,j)] for (i,j) in A), GRB.MAXIMIZE)



In [ ]:
#Constraints

# Customers can only select one choice
ChoiceConstrs = m.addConstrs((gp.quicksum(y[(i,j)] for j in candy) <= 1 for i in customer), 
                                      name='choiceConstrs')

# Find the maximum utility for any given price p
ChoiceConstrs1 = m.addConstrs( (auxU[(i,j)] == cum[i][j] - f - p*(j+1) for i,j in A), name="aux_utility")
ChoiceConstrs2 = m.addConstrs( (auxUMax[i] >= auxU[(i,j)] for i,j in A), name="aux_utility_max")

# Choice must maximize utility
UtilityConstrs = m.addConstrs( ((cum[i][j]-f - p*(j+1))*y[(i,j)] >= auxUMax[i] - M*(1-y[(i,j)]) for i,j in A),
                                      name="utilityConstrs")

# Customer purchase only if utility is non-negative
PositiveConstrs = m.addConstrs( ((cum[i][j]-f - p*(j+1))*y[(i,j)] >= 0 for i,j in A),
                                      name="positiveConstrs")

Solve the model

In [ ]:
# Run optimization engine
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 84 rows, 86 columns and 240 nonzeros
Model fingerprint: 0xabe10e81
Model has 80 quadratic objective terms
Model has 80 quadratic constraints
Variable types: 46 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  QMatrix range    [1e+00, 1e+01]
  QLMatrix range   [9e-01, 9e+00]
  Objective range  [2e+00, 6e+01]
  QObjective range [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+01]
  RHS range        [9e-01, 8e+00]
  QRHS range       [1e+01, 1e+01]
Presolve added 50 rows and 0 columns
Presolve removed 0 rows and 30 columns
Presolve time: 0.01s
Presolved: 214 rows, 136 columns, 1025 nonzeros
Variable types: 96 continuous, 40 integer (40 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective 7.978833e+02, 41 i

Examine outputs

In [ ]:
# check the optimal solution
print("The entry fee is $", round(f.x,2), ", optimal price is $",round(p.x,2), "and the associated profit is $", round(m.ObjVal,2))

The entry fee is $ 3.31 , optimal price is $ 0.4 and the associated profit is $ 107.57


In [ ]:
# print optimal puchase by customers

print("\033[1m Optimal purchase decisions by customers")
print("------------------------------------------\n")
# loop through all customers
for i in customer:
  for j in candy:
    # if customer i purchases j pieces
    if y[(i,j)].x >0:
      print("Customer",i+1,"purchases:", j+1, "pieces", end="    ")
  print("\n")  


 Optimal purchase decisions by customers
------------------------------------------

Customer 1 purchases: 9 pieces    



Customer 3 purchases: 8 pieces    

Customer 4 purchases: 9 pieces    



In [ ]:
# print detailed utilities by customers

print("\033[1m Utilities of purchase decisions by customers\033[0m")
print("------------------------------------------\n")
# loop through all customers
for i in customer:
  print("customer",i+1,"\n")
  for j in candy:
    if y[(i,j)].x >0:
      print(j+1, "pieces:", round(cum[i][j],2), "-", round(f.x,2), "-", round(p.x*(j+1),2), "=", round(cum[i][j]-f.x-p.x*(j+1),2),"*",end="")
    else:
      print(j+1, "pieces:", round(cum[i][j],2), "-", round(f.x,2), "-", round(p.x*(j+1),2), "=", round(cum[i][j]-f.x-p.x*(j+1),2),end="")
    print("\n")  
  print("------------------------------------------\n")


 Utilities of purchase decisions by customers
------------------------------------------

customer 1 

1 pieces: 1.24 - 3.31 - 0.4 = -2.47

2 pieces: 2.27 - 3.31 - 0.8 = -1.84

3 pieces: 3.16 - 3.31 - 1.2 = -1.35

4 pieces: 3.96 - 3.31 - 1.6 = -0.95

5 pieces: 4.73 - 3.31 - 2.0 = -0.58

6 pieces: 5.39 - 3.31 - 2.4 = -0.32

7 pieces: 5.98 - 3.31 - 2.8 = -0.13

8 pieces: 6.49 - 3.31 - 3.2 = -0.02

9 pieces: 6.91 - 3.31 - 3.6 = 0.0 *

10 pieces: 7.26 - 3.31 - 4.0 = -0.05

------------------------------------------

customer 2 

1 pieces: 0.92 - 3.31 - 0.4 = -2.79

2 pieces: 1.77 - 3.31 - 0.8 = -2.34

3 pieces: 2.46 - 3.31 - 1.2 = -2.05

4 pieces: 3.04 - 3.31 - 1.6 = -1.87

5 pieces: 3.54 - 3.31 - 2.0 = -1.77

6 pieces: 3.97 - 3.31 - 2.4 = -1.74

7 pieces: 4.33 - 3.31 - 2.8 = -1.78

8 pieces: 4.65 - 3.31 - 3.2 = -1.86

9 pieces: 4.91 - 3.31 - 3.6 = -2.0

10 pieces: 5.13 - 3.31 - 4.0 = -2.18

------------------------------------------

customer 3 

1 pieces: 1.27 - 3.31 - 0.4 = -2.44

2 pie

#Conclusion

The optimal two-part tariff shows that the profit can be significantly improved over a single price strategy. With two-part tariff, the optimal price reaches \$107,000. 

A key take away of the above example is that a non-linear pricing strategy can help to extract more consumer surplus than a single pricing strategy. An important technique is to use integer variables to denote whether a customer buys a certain pieces of candies, and then leverage this information to compute the associated profit. 

##  References

[1] Gurobi python reference. https://www.gurobi.com/documentation/

[2] This notebook is developed by Yimin Wang (yimin_wang@asu.edu)